## Look at the cracks, the postprocessed data and the visual analysis results:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import skeletonize
from skimage.morphology import medial_axis

import fracsim.measure.isolines as isolines_calculator

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "lmodern",
        "font.size": 11.0,
        # "font.family": "serif",
        # "font.serif": "Palatino",
        "axes.titlesize": "medium",
        "figure.titlesize": "medium",
        "text.latex.preamble": "\\usepackage{amsmath}\\usepackage{amssymb}\\usepackage{siunitx}",
        "savefig.dpi": 300,
        "figure.dpi": 300,
    }
)

df = pd.read_csv("results/analysis_results.csv", index_col=0)

for index, row in df.iterrows():
    OP_bw = row["OP_01_bw"]
    OP_bw = np.load(OP_bw)
    OP_bw = OP_bw > 0.5
    OP_01 = np.load(row["OP_01"])
    skel, distance = medial_axis(OP_bw, return_distance=True)
    skeleton = skeletonize(OP_bw)
    dist_on_skel = distance * skeleton

    summed_width = np.sum(dist_on_skel)
    print(f"summed_width: {summed_width}")
    print(f"width_variance: {np.var(dist_on_skel)}")

    plt.imshow(dist_on_skel, cmap='magma')
    plt.colorbar()

    plt.contour(OP_01, [0.98], colors='w')
    plt.show()

In [ ]:
for index, row in df.iterrows():
    # Draw the original image
    OP_01 = row["OP_01_bw"]
    arr = np.load(OP_01)
    #plt.imshow(arr, cmap=plt.cm.gray)
    # plt.show()

    # Draw the skeleton
    skeleton = np.load(row["skeleton"])
    skeleton = arr - skeleton
    plt.imshow(skeleton, cmap=plt.cm.gray)
    plt.xlabel("x [\\unit{\\milli\meter}]")
    plt.ylabel("y [\\unit{\\milli\meter}]")
    # Change axis from pixel to mm which means an offset of -250 in y direction and 50 in x direction
    # The mesh goes from -250 to 250 in y direction and from 50 to 850 in x direction
    plt.xticks([0, 100, 200, 300, 400, 500, 600, 700, 800], [50, 150, 250, 350, 450, 550, 650, 750, 850])
    plt.yticks([0,100,200,250,300,400,500], [250,150,50,0,-50,-150,-250])
    plt.savefig(f"results/skeletons/skeleton_{index}.pdf", bbox_inches='tight')
    plt.show()

    print('Skeleton length:', row["skeleton_length"])


## Plot Isolines

In [ ]:
for index, row in df.iterrows():
    OP_bw = np.load(row["OP_01_bw"])
    plt.imshow(OP_bw, cmap=plt.cm.gray)
    OP_01_isolines = isolines_calculator.isoline(OP_bw, iso_value=row["iso_value"])
    for isoline in OP_01_isolines:
        plt.plot(isoline[:, 1], isoline[:, 0], linewidth=2, color='red')
    plt.show()
    
    print("The total curvature is:", row["total_curvature"])
    for smoothing_cyle in range(1, row["curvature_smoothing_cycles"]+1):
        curvature_smoothed = row[f"curvature_smoothed_{smoothing_cyle}"]
        print(f"The curvature after {smoothing_cyle} smoothing cycles is:", curvature_smoothed)

for index, row in df.iterrows():
    # select where std_lambda == 5.05e8
    sub_df = df[df["std_lambda"]==5.05e8]
    strides = [1,3,5,10,25]
    for stride in strides:
        plt.plot(sub_df["lengthscale_lambda"])

## Plot single (lambda, phase field)

In [ ]:
import pyvista as pv
import os

vtk_files = df["vtk_structured"]
n_vtk_files = len(vtk_files)
n_rows = 1#n_vtk_files // 5 + 1
n_cols = 2

scalars = ["LAMBDA_S", "OP"]

# Create a plotter with multiple render windows
plotter: pv.Plotter = pv.Plotter()#shape=(n_rows, n_cols), border=False, window_size=(800, 1080))
# Iterate over the VTK files
for i in range(n_vtk_files):
    # check if index is in the data frame index
    if i not in df.index:
        continue
    # Check if the file exists
    if os.path.isfile(vtk_files[i]):
        # Load the VTK file
        mesh = pv.read(vtk_files[i])
        # Set the active render window
        for j, scalar in enumerate(scalars):
            # plotter.subplot(0, j)
            # Add the mesh to the plotter
            plotter.add_mesh(mesh, scalars=scalar)
            # plotter.reset_camera_clipping_range()
            plotter.view_xy()
            # set correct scaling in x and y direction
            plotter.set_scale(1, 2, 1)
            # Set the colorbar position to the right of the plot
            plotter.add_scalar_bar(title=scalar, vertical=True, width=0.5, height=0.8, position_x=0.1, position_y=0.1)
            # zoom in
            #plotter.camera.zoom(1.5)
            # Show the plot
            plotter.show()
    else:
        print(f"File {vtk_files[i]} does not exist.")

## Plot Ensemble (Lambda, Phase Field)

In [ ]:
import pyvista as pv
import os

vtk_files = df["vtk_structured"]
n_vtk_files = len(vtk_files)
n_rows = n_vtk_files // 5 + 1
n_cols = 5

scalar = "LAMBDA_S"
#scalar = "OP"

# Create a plotter with multiple render windows
plotter: pv.Plotter = pv.Plotter(shape=(n_rows, n_cols), border=False, window_size=(800, 1080))
# Iterate over the VTK files
for i_row in range(n_rows):
    for i_col in range(n_cols):
        if(i_row*n_cols + i_col < n_vtk_files):
            # check if index is in the data frame index
            if i_row*n_cols + i_col not in df.index:
                continue
            # Check if the file exists
            if os.path.isfile(vtk_files[i_row*n_cols + i_col]):
                # Load the VTK file
                mesh = pv.read(vtk_files[i_row*n_cols + i_col])
                # Set the active render window
                plotter.subplot(i_row, i_col)
                # Add the mesh to the plotter
                plotter.add_mesh(mesh, scalars=scalar)
                plotter.reset_camera_clipping_range()
                plotter.view_xy()
                # zoom in
                plotter.camera.zoom(1.5)
            else:
                print(f"File {vtk_files[i_row*n_cols + i_col]} does not exist.")
# Show the result
plotter.show()